In [ ]:
#!/usr/bin/env python
# coding: utf-8


import os
import pandas as pd
import numpy as np

import openpyxl

#from datetime import datetime

### Definition of functions

In [73]:
# define incorrect values and link them with the correct ones (empty string) in order to get proper join later

def replace_value(string: str):
    mapping_dict = {
            ".": "",
            ",": ""
            }
    
    for key, value in mapping_dict.items():
        if key in string:
            updated_string = string.replace(key, value)                
            return updated_string
        else:
            return string

In [ ]:
# Function reads input file

def read_file(input_file):
    file_in = os.path.join(config_file.loc["input_folder","Value"],input_file)
    df = pd.read_csv(file_in,encoding="utf-8",delimiter=config_file.loc["csv_delimiter","Value"], dtype=str) 

    return df  

In [ ]:
# update_value function is called + rows with incorrect data are removed from df

def update_dataframe(input_file):
    input_file["Costcenter"] = input_file["Costcenter"].apply(replace_value)
    
    list_of_incorrect_strings = ["Registration Pool", "License Pool"]
    condition = input_file['Costcenter'].str.contains('|'.join(list_of_incorrect_strings), na = False)
    
    return input_file[~condition]

In [ ]:
# Merge input files

def merge_file(input_file):
    
    temp_size = input_file.shape[0]
    df = pd.merge(input_file,
            cost_centers,
            left_on="Costcenter",
            right_on="Costcenter",
            how="left")
    assert input_file.shape[0] == temp_size
    
    return df

### Input settings

In [ ]:
new_line = '\n'

try:
    config_file = pd.read_excel("configurations.xlsx").set_index("What")
    
    year = config_file.loc["year","Value"]
    month = config_file.loc["month","Value"]

    final_file_name = f"Active Equipment Report_OSD-Gesamt_{year}_{month}.csv"
    output_directory_name = f"M_{year}_{month}"
    
except FileNotFoundError:
    print("File 'configurations' does not exists in the directory where the script is located.")
    
# config_file

### Get input files

In [ ]:
# ## Read data (Costcenters)
# ### Get catalogue for joins

try:
    cost_centers = pd.read_excel(config_file.loc["cost_centers","Value"])
    cost_centers = cost_centers.rename(columns={"KOSTENSTELLE_Katalog":"Costcenter",
                                                "ORG_EBENE_1_Katalog":"Org-Ebene_1",
                                                "ORG_EBENE_2_Katalog":"Org-Ebene_2",
                                                "ORG_EBENE_3_Katalog":"Org-Ebene_3"})

    cost_centers = cost_centers[["Costcenter","Org-Ebene_1","Org-Ebene_2","Org-Ebene_3"]]

    cost_centers = cost_centers.astype({"Costcenter": str})
    cost_centers["Costcenter"] =  cost_centers["Costcenter"].apply(replace_value)

except FileNotFoundError:
    print(f"Path {config_file.loc['cost_centers','Value']} is not valid.")

# cost_centers

,Costcenter,Org-Ebene_1,Org-Ebene_2,Org-Ebene_3
0,26040023,ISP,ISP,NaN
1,26040031,ISP,ISP,NaN
2,26040428,ISP,ISP,NaN
3,26041192,ISP,ISP,NaN
4,26043400,ISP,ISP,NaN
...,...,...,...,...
4878,T8X4130017,DTBS,Mexiko,NaN
4879,T8X1000061,DTBS,Mexiko,NaN
4880,T8X1000002,DTBS,Mexiko,NaN
4881,T8X1000018,DTBS,Mexiko,NaN


In [ ]:
# ### Get all files inside folder

all_files = os.listdir(config_file.loc["input_folder","Value"])

if not pd.isna(config_file.loc["input_folder","Value"]):
    all_files = [i for i in all_files if config_file.loc["input_file_name_mask","Value"] in i]
else:
    print(f"Directory {config_file.loc['input_folder','Value']} is empty.")

# all_files

### Main script

In [ ]:
list_of_dataframes = []

for file in all_files:
    adjusted_file = update_dataframe(read_file(file))    
    merged_file = merge_file(adjusted_file)
    
    list_of_dataframes.append(merged_file)

df_final = pd.concat(list_of_dataframes, ignore_index=True, sort=False)

df_final

,PortfolioCluster,PortfolioService,Equipment_Name,SerialNumber,HW_Type,ServiceID,Equipment_ID,ParentEquipmentID,Price,OrderDate,...,AccountingArea,Costcenter,Username,UserAccount,UserEmail,UserEmploymentStatus,Location,Org-Ebene_1,Org-Ebene_2,Org-Ebene_3
0,Collaboration Services,FBDX Groupshare,g_08993277$,NaN,NaN,8993277,FX#2121719,NaN,0.93 EUR,NaN,...,2,D4AD370210,NaN,NaN,NaN,Employee,"DE,53227,Bonn,Landgrabenweg 147",TDG - ZW,ZW,BQW
1,Collaboration Services,FBDX Groupshare,g_0A328407$,NaN,NaN,0A328407,FX#2010123,NaN,0.93 EUR,NaN,...,2,D4AD210760,NaN,NaN,NaN,Employee,"DE,53227,Bonn,Oberkasseler Straße 2",GK,GK-CRM&Kom,GK-CRM&Kom
2,Collaboration Services,FBDX Groupshare,g_0A271308$,NaN,NaN,0A271308,FX#2042693,NaN,0.93 EUR,NaN,...,2,D4AD210760,NaN,NaN,NaN,Employee,"DE,53227,Bonn,Oberkasseler Straße 2",GK,GK-CRM&Kom,GK-CRM&Kom
3,Collaboration Services,FBDX Groupshare,g_09087052$,NaN,NaN,9087052,FX#2060899,NaN,0.93 EUR,NaN,...,2,D4AD210761,NaN,NaN,NaN,Employee,"DE,53227,Bonn,Landgrabenweg 151",GK,GK-CRM&Kom,GK-CRM&Kom
4,Collaboration Services,FBDX Groupshare,g_09206120$,NaN,NaN,9206120,FX#2052432,NaN,0.93 EUR,NaN,...,2,D4AD210761,NaN,NaN,NaN,Employee,"DE,53227,Bonn,Landgrabenweg 151",GK,GK-CRM&Kom,GK-CRM&Kom
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3601556,Virtual Workplace,VDS One Time Pass (OTP),SID: T0006e0902,NaN,NaN,T0006e0902,1010162786,NaN,NaN,28.06.2019 08:48,...,8108,T1A0002090,NaN,NaN,NaN,Freelancer,"DE,40547,Düsseldorf,Am Seestern 3",DTBS,PU TC Services,NaN
3601557,Virtual Workplace,VDS One Time Pass (OTP),SID: T0006e0926,NaN,NaN,T0006e0926,1010162812,NaN,NaN,28.06.2019 08:51,...,8108,T1A0002090,NaN,NaN,NaN,Freelancer,"DE,40547,Düsseldorf,Am Seestern 3",DTBS,PU TC Services,NaN
3601558,Virtual Workplace,VDS One Time Pass (OTP),SID: T000d62e34,NaN,NaN,T000d62e34,1016634984,NaN,NaN,07.07.2022 08:24,...,8108,T1A0001701,NaN,NaN,NaN,Freelancer,"DE,60528,Frankfurt am Main,Hahnstraße 43 d",DTBS,PU TC Services,NaN
3601559,Virtual Workplace,VDS One Time Pass (OTP),SID: T000ba1b1d,NaN,NaN,T000ba1b1d,1014886624,NaN,NaN,02.12.2021 14:59,...,8108,T1A0001887,NaN,NaN,NaN,Freelancer,"DE,60528,Frankfurt am Main,Hahnstraße 43 d",DTBS,PU TC Services,NaN


In [ ]:
# Identification of missing costcenters
    
mask = df_final["Org-Ebene_1"].isna()    
df_missing_costcenters = df_final[mask][["Costcenter"]].drop_duplicates()
df_missing_costcenters

,Costcenter
214025,8000100
217749,SP11727
219114,D1AD150550
219192,D1AKK62561
219262,D1AKW63812
221060,D1ATS32122
221061,D1ATV32240
221062,D1ATA32122
221066,D1ATK32130
221067,D1ATV32340


In [ ]:
# Determine if there are missing costcenters, if not, create folder and all output files.
# If there are missing costcenters, append them to Kostenstellenkatalog.xlsx  

if df_missing_costcenters.shape[0] == 0:
     
    output_path = os.path.join(config_file.loc['output_folder','Value'], output_directory_name) 
    
    # Create the directory if not exists 
    if not os.path.exists(output_path):
        os.makedirs(output_path) 
        print(f"Directory {output_directory_name} has been created.", new_line) 
    else:
        print(f"Directory {output_directory_name} already exists. Please replace the month and eventuell also year values in the configurations file.", new_line) 
        print("Ending script...")
        raise SystemExit
    
    print(f"Creating files in {output_directory_name}...", new_line)
    
    for index, dataframe in enumerate(list_of_dataframes):
        dataframe.to_csv(os.path.join(output_path ,all_files[index].replace(".csv","_out.csv")), index = False)
        print("File", all_files[index].replace('.csv','_out.csv'), "has been created.")
        # break # Remove after testing!
    
    df_final.to_csv(os.path.join(output_path, final_file_name), index = False)
    print(new_line)
    print("File", final_file_name, "has been created.")

else:
    # with pd.ExcelWriter(config_file.loc["cost_centers","Value"],mode="a",engine="openpyxl",if_sheet_exists="overlay") as writer:
    #     df_missing_costcenters.to_excel(writer, sheet_name="Kostenstellenliste",header=None, startrow=writer.sheets["Kostenstellenliste"].max_row,index=False)
    df_missing_costcenters.to_csv(os.path.join(config_file.loc["cost_centers_folder","Value"], "Fehlende_Werte.csv"), header = False, index = False)
    
    print("Files have not been created, missing costcenters have been added to 'Fehlende_Werte.xlsx'.")
    print("Run the script again after adding all respective values.", new_line)


Files have not been created, missing costcenters have been added to 'Kostenstellenkatalog'.
Run the script again after adding all respective values. 



In [83]:
# # Testing

# parent_dir = config_file.loc['output_folder','Value']
    
# output_path = os.path.join(config_file.loc['output_folder','Value'], output_directory_name) 
    
# # Create the directory if not exists 
# if not os.path.exists(output_path):
#     os.makedirs(output_path) 
#     print(f"Directory {output_directory_name} created.", new_line) 
# else:
#     print(f"Directory {output_directory_name} already exists. Please replace the month and eventuell also year values in the configurations file", new_line) 
#     print("Ending script...")
#     quit()

Directory M_2022_10 created. 



In [84]:
# # Testing

# for dataframe in list_of_dataframes:
#     dataframe.to_csv(os.path.join(output_path,file.replace(".csv","_out.csv")), index = False)
#     print("File", file.replace('.csv','_out.csv'), "has been created.")
#     break

# print(new_line)
# df_final.to_csv(os.path.join(output_path, final_file_name), index = False)
# print("File", final_file_name, "has been created.")

File Active Equipment Report_8108_2022_10_out.csv has been created.


File Active Equipment Report_OSD-Gesamt_2022_10.csv has been created.
